In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx  
from scipy.spatial.distance import pdist, squareform  
from tqdm import tqdm  
from pyproj import Transformer, CRS  
# import seaborn as sns
# import regex as re

# from sklearn.neighbors import KernelDensity
# pd.options.display.max_columns = None
# from sklearn import preprocessing

In [2]:

from sklearn.model_selection import train_test_split #, cross_validate 
# from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression #, Ridge  , LogisticRegression , QuantileRegressor
from sklearn.ensemble import GradientBoostingRegressor # , RandomForestRegressor , HistGradientBoostingRegressor 
from sklearn.metrics import mean_absolute_percentage_error , r2_score, mean_squared_error
# from xgboost import XGBRegressor
# from geopy.geocoders import Nominatim
# from concurrent.futures import ThreadPoolExecutor

###########################################################################################################################
from gensim.models import Word2Vec
# from gensim.scripts.glove2word2vec import glove2word2vec
# from gensim.models import KeyedVectors
# import re
# from sklearn.decomposition import PCA
# from matplotlib import pyplot as plt
# from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import json
import plotly.express as px
import networkx as nx
from geopy.distance import geodesic
from scipy.spatial import cKDTree

# Load the real estate dataset for Mashhad, Iran

In [3]:
data= pd.read_excel('Mashhad_Real_Estate_Data.xlsx')
data.head()

,type,area_sq_m,latitude,longitude,age_years,floor_number,number_of_bedrooms,price,region,elevator,...,stone_façade,garden,direction,furnished,renovated,closet,em_mos,repainted,property_document_type,utilities_included
0,0,73.0,36.373776,59.475820,11,1,2,2200000000,12,1,...,0,0,0,0,0,0,0,0,1,4
1,0,100.0,36.327670,59.548380,7,1,2,4800000000,2,1,...,0,0,0,0,0,0,0,0,0,3
2,0,140.0,36.310605,59.504477,0,2,2,10200000000,9,1,...,0,1,1,0,0,0,0,0,1,4
3,0,65.0,36.400540,59.388382,6,4,1,870000000,12,0,...,0,0,0,0,0,1,1,1,2,0
4,0,200.0,36.373776,59.475820,0,0,3,5350000000,12,1,...,0,0,1,0,0,0,0,0,1,4


In [4]:
data.columns

Index(['type', 'area_sq_m', 'latitude', 'longitude', 'age_years',
       'floor_number', 'number_of_bedrooms', 'price', 'region', 'elevator',
       'parking', 'storage', 'shop', 'balcony', 'full_amenities', 'parquet',
       'ceramic_flooring', 'stone_façade', 'garden', 'direction', 'furnished',
       'renovated', 'closet', 'em_mos', 'repainted', 'property_document_type',
       'utilities_included'],
      dtype='object')

In [5]:
data.shape

(44781, 27)

In [13]:
# Check if there are any NaN values in the DataFrame
has_nan = data.isnull().any().any()
print(has_nan)  # True if there are any NaN values

False


## Filter the data to focus on properties within a specific geographic region of Mashhad
### The latitude and longitude ranges likely correspond to the city's central or most relevant area

In [14]:
filtered_data = data[(data['longitude'] >= 59.45) & (data['longitude'] <= 59.7) & 
                 (data['latitude'] >= 36.2) & (data['latitude'] <= 36.4)]
filtered_data.shape

(44670, 27)

Randomly sample 10,000 data points from the filtered dataset

This creates a smaller, manageable dataset for analysis while maintaining diversity and reducing potential bias from any inherent order in the original data

In [15]:
np.random.seed(42)
shuffle_indices = np.random.choice(np.arange(filtered_data.shape[0]), size=10000, replace=False,)
df = filtered_data.iloc[shuffle_indices].reset_index(drop=True)
df['id'] = df.index  # Add this line to create a unique identifier for each house  
len(df) #, df.head(2)

10000

Visualize the spatial distribution of the selected properties using a scatter plot

In [9]:
import plotly.express as px
px.scatter(df , x="longitude", y="latitude",  width=400, height=400)#.write_image("img/spatial_distribution.png")

Creates a NetworkX graph of houses in Mashhad

In [ ]:
# import numpy as np  
# import pandas as pd  
# import networkx as nx  
# from pyproj import CRS, Transformer  
# from scipy.spatial.distance import pdist, squareform  
# from tqdm import tqdm  # For progress bar  

# def create_mashhad_house_graph(df, distance_threshold_meters=200):  
#     """  
#     Creates a NetworkX graph of houses in Mashhad, using UTM projection.  

#     Args:  
#         df: Pandas DataFrame with 'latitude', 'longitude', and 'type' columns.  
#         distance_threshold_meters: Maximum distance (in meters) between houses to be connected.  

#     Returns:  
#         A NetworkX graph, or None if an error occurs.  
#     """  
#     # Define input and output CRS (Coordinate Reference Systems)  
#     in_crs = CRS.from_epsg(4326)  # WGS 84 (lat/long)  
#     out_crs = CRS.from_epsg(32640)  # UTM zone 40N  

#     transformer = Transformer.from_crs(in_crs, out_crs, always_xy=True)  

#     # Initialize an empty graph  
#     G = nx.Graph()  

#     # Transform coordinates to UTM  
#     coords_utm = np.array([transformer.transform(lon, lat) for lat, lon in zip(df['longitude'], df['latitude'])])  # longitude, then latitude  

#     # Calculate Euclidean distances in meters  
#     distances = squareform(pdist(coords_utm))  

#     # Add nodes to the graph with their respective attributes from the DataFrame  
#     nodes = {}  
#     for index, row in df.iterrows():  
#         G.add_node(index, pos=(row['latitude'], row['longitude']), Type=row['type'])  # Adjust attributes if needed  
#         nodes[index] = len(nodes)  

#     # Create edges based on the distance threshold  
#     num_houses = len(nodes)  
#     node_list = list(nodes.keys())  
#     for i in tqdm(range(num_houses), desc="Building graph"):  
#         for j in range(i + 1, num_houses):  
#             index1 = node_list[i]  
#             index2 = node_list[j]  
#             if distances[i, j] <= distance_threshold_meters:  
#                 G.add_edge(index1, index2, weight=distances[i, j])  

#     return G

# # Example usage:  
# # df = pd.DataFrame({...})  # Your DataFrame with required columns  
# # graph = create_mashhad_house_graph(df, distance_threshold_meters=500)  
# # print(f"Number of edges in the graph: {graph.number_of_edges()}")

In [2]:
# # Example usage  
# G = create_mashhad_house_graph(df)  
# num_edges = G.number_of_edges()  
# print(f"Number of edges in the graph: {num_edges}")
# for node in list(G.nodes(data=True))[:5]:  # Display the first 5 nodes and their attributes  
#     print(node)

In [9]:
import networkx as nx  
import pandas as pd  
from sklearn.preprocessing import StandardScaler  
from scipy.spatial import cKDTree  
import numpy as np  
from tqdm import tqdm  

def create_graph_from_dataframe(df, distance_threshold):  
    """  
    Create a graph from a DataFrame of property features, with edges based on  
    geographic proximity and weighted by property attributes.  

    Parameters:  
    - df: pd.DataFrame containing property features.  
    - distance_threshold: float, the distance threshold in meters for creating edges.  

    Returns:  
    - G: networkx.Graph, the generated graph with nodes and weighted edges.  
    """  
    # Separate numerical and binary features  
    numeric_features = ['area_sq_m', 'age_years', 'floor_number', 'number_of_bedrooms']  
    binary_features = ['elevator', 'parking', 'storage', 'balcony', 'parquet',   
                      'ceramic_flooring', 'stone_façade', 'garden', 'renovated']  
    
    # Normalize numeric features  
    scaler = StandardScaler()  
    df[numeric_features] = scaler.fit_transform(df[numeric_features])  

    # Build spatial index with cKDTree  
    coordinates = df[['latitude', 'longitude']].to_numpy()  
    tree = cKDTree(coordinates)  

    # Initialize graph  
    G = nx.Graph()  

    # Add nodes with attributes using the new 'id' column  
    for i, row in df.iterrows():  
        G.add_node(row['id'], **row.to_dict())  # Use the 'id' as the node identifier  

    # Find pairs of nodes within the threshold distance  
    pairs = tree.query_pairs(distance_threshold / 111000)  # Approximate meter-to-degree conversion  

    # Define a function to compute edge weight  
    def calculate_weight(df, idx1, idx2):  
        # no edge if the types are different  
        if df.loc[idx1, 'type'] != df.loc[idx2, 'type']:  
            return 0  
        geo_distance = np.linalg.norm(coordinates[idx1] - coordinates[idx2])  
        num_distance = np.linalg.norm(df.loc[idx1, numeric_features] - df.loc[idx2, numeric_features])  
        binary_similarity = np.sum(df.loc[idx1, binary_features] == df.loc[idx2, binary_features])  
        
        # Weight calculation (adjust formula as needed)  
        weight = (1 / (1 + geo_distance)) * (1 + binary_similarity) / (1 + num_distance)  
        return weight  

    # Add edges with weights  
    for idx1, idx2 in tqdm(pairs):  
        weight = calculate_weight(df, idx1, idx2)  
        if weight != 0:  
            G.add_edge(df.at[idx1, 'id'], df.at[idx2, 'id'], weight=weight)  # Use ids for edges  

    return G  

In [10]:
G = create_graph_from_dataframe(df, distance_threshold=1000) 

100%|██████████| 3545375/3545375 [1:52:21<00:00, 525.88it/s]  


In [11]:
def is_complete_graph(G):
    n = G.number_of_nodes()
    if n < 2:
        return True  # A graph with 0 or 1 node is considered complete
    max_edges = n * (n - 1) // 2
    print(G.number_of_edges(), max_edges)
    return G.number_of_edges() == max_edges

In [12]:
print("G is complete:", is_complete_graph(G)) 

3296629 49995000
G is complete: False


In [18]:
# len(G.edges()), G.nodes[0]

In [ ]:
# import pickle

# with open('D:/git/mamintoosi/Deepwalk-HPP/var/graph-dist-1000.pkl', 'wb') as f:
#     pickle.dump(G, f)

In [6]:
import pickle
with open('D:/git/mamintoosi/Deepwalk-HPP/var/graph-dist-1000.pkl', 'rb') as f:
    G = pickle.load(f)

In [12]:
# nx.write_sparse6(G, "mashhad_graph.s6")

In [ ]:
# G = nx.read_sparse6("var/mashhad_graph_dist-1000.s6")

In [19]:
df.head(1)

,type,area_sq_m,latitude,longitude,age_years,floor_number,number_of_bedrooms,price,region,elevator,...,garden,direction,furnished,renovated,closet,em_mos,repainted,property_document_type,utilities_included,id
0,0,-0.342552,36.290244,59.507194,-0.932219,-0.238805,-0.457341,4200000000,9,1,...,0,1,0,0,0,0,0,1,4,0


In [21]:
# len(G.edges()), G.nodes[0]

## Function: Fit and Evaluate a Regression Model

This function takes a regression model and training/testing data as input, fits the model, and evaluates its performance using several metrics. Here's a breakdown:

*   **Model Fitting:** The function first fits the provided `model` to the training data (`X_train`, `y_train`). This step involves the model learning patterns from the training data to make predictions.
*   **Predictions:** It then uses the fitted model to predict the target variable (`y_pred`) for the test data (`X_test`).
*   **Performance Evaluation:** The function calculates several metrics to assess the model's performance:
    *   **R-squared (R2):** Measures the proportion of variance in the target variable explained by the model. Higher R2 values indicate better fit. 
    *   **Mean Absolute Percentage Error (MAPE):** Measures the average percentage difference between the predicted and actual values. Lower MAPE values indicate better accuracy.
    *   **Mean Squared Error (MSE):** Measures the average squared difference between the predicted and actual values. Lower MSE values indicate better accuracy.
    *   **Range Accuracy:** This metric is defined within the function. It calculates the proportion of predictions that fall within a certain tolerance range of the actual values. The tolerance is set by the `tolerance` parameter (default is 0.2, or 20%). 
*   **Visualization:** The function creates a scatterplot of the actual target values (`y_test`) against the predicted values (`y_pred`). This visualization helps to visually assess the model's performance and identify any potential patterns or outliers.

## Function: Random Walk in a Graph

This function performs a random walk on a graph `G`, starting from a given node `start` and continuing for a specified `length`. Here's a breakdown:

*   **Initialization:**
    *   `walk = [str(start)]`: The function initializes a list called `walk` to store the nodes visited during the walk. It starts by adding the `start` node to the list as a string.
*   **Iteration:**
    *   `for i in range(length):`: The function iterates `length` times, representing the steps in the random walk.
        *   `neighbors = [node for node in G.neighbors(start)]`: It gets a list of neighbors of the current node `start` from the graph `G`.
        *   `next_node = np.random.choice(neighbors, 1)`: It randomly selects one of the neighbors as the `next_node` using the `np.random.choice` function.
        *   `walk.append(str(next_node))`: The `next_node` is added to the `walk` list as a string.
        *   `start = next_node`: The `next_node` becomes the new `start` node for the next step in the walk.
*   **Return:**
    *   `return walk`: The function returns the `walk` list, which contains the sequence of nodes visited during the random walk.

In [7]:
import numpy as np  

def random_walk(start, length):  
    walk = [str(start)]  # Starting node  

    for i in range(length):  
        # Get the neighbors of the current node  
        neighbors = [node for node in G.neighbors(start)]  
        
        # Check if the node has neighbors  
        if len(neighbors) == 0:  
            next_node = start  # If no neighbors, stay in the same node  
        else:  
            next_node = np.random.choice(neighbors, 1)[0]  # Choose a neighbor at random  
        
        walk.append(str(next_node))  
        start = next_node  # Move to the next node  
    
    return walk

In [23]:
nx.is_connected(G)

False

After sampling the dataset, we created a graph of 10,000 properties for the random walk algorithm. To ensure applicability, the graph must be connected, so we identified the largest connected subgraph. The process for extracting this component is described below:

In [24]:
import networkx as nx

# Find the largest connected component
largest_cc = max(nx.connected_components(G), key=len)

# Create a subgraph from the largest connected component
largest_subgraph = G.subgraph(largest_cc).copy()

# If you want to remove the disconnected nodes from the original graph
# G = largest_subgraph
# nx.is_connected(G)
nx.is_connected(largest_subgraph)

True

In [25]:
# Get the nodes in the largest connected component  
nodes_in_largest_cc = set(largest_subgraph.nodes())  

# Use boolean indexing to filter the DataFrame  
df_filtered = df[df.index.isin(nodes_in_largest_cc)] 

In [26]:
# Use the largest connected component graph  
# largest_subgraph = G.subgraph(largest_cc).copy()  

# You can now work with largest_subgraph for graph-related operations, such as:  
print(f"Number of nodes in the largest subgraph: {largest_subgraph.number_of_nodes()}")  
print(f"Number of edges in the largest subgraph: {largest_subgraph.number_of_edges()}")  

# And you can work with df_filtered for DataFrame-related operations:  
print(df_filtered.shape)

Number of nodes in the largest subgraph: 8949
Number of edges in the largest subgraph: 3277632
(8949, 28)


In [27]:
#Verify the number of rows:  
print(f"Original DataFrame shape: {df.shape}")  
print(f"Filtered DataFrame shape: {df_filtered.shape}")  
G.number_of_nodes(), G.number_of_edges(), \
largest_subgraph.number_of_nodes(), largest_subgraph.number_of_edges()

Original DataFrame shape: (10000, 28)
Filtered DataFrame shape: (8949, 28)


(10000, 3296629, 8949, 3277632)

In [28]:
px.scatter(df_filtered , x="longitude", y="latitude",  width=400, height=400)#.write_image("img/spatial_distribution.png")

In [21]:
# df_filtered.head(5)

In [30]:
# G.nodes, largest_subgraph.nodes

In [8]:
%%time
from tqdm import tqdm  

def generate_random_walks(subgraph, num_walks=80, walk_length=10):  
    """  
    Generate random walks for each node in the given subgraph.  

    Parameters:  
    - subgraph: The graph from which to generate random walks.  
    - num_walks: The number of random walks to generate for each node.  
    - walk_length: The length of each random walk.  

    Returns:  
    - A list of random walks.  
    """  
    print(subgraph.number_of_nodes())
    walks = []  
    for node in tqdm(subgraph.nodes, desc="Nodes"):  
        for _ in range(num_walks):  
            walks.append(random_walk(node, walk_length))  
    return walks  

CPU times: total: 0 ns
Wall time: 0 ns


In [9]:
# # Example usage:  
# # Assuming 'largest_subgraph' is your subgraph, you can call the function like this:  
# num_walks = 80  # Number of walks per node  
# walk_length = 10  # Length of each walk  
# walks = generate_random_walks(largest_subgraph, num_walks, walk_length)  

# # Print the first random walk  
# print(walks[0])

In [25]:
import gzip
import pickle

# # Save walks to a compressed file
# with gzip.open('../walks.pkl.gz', 'wb') as f:
#     pickle.dump(walks, f)


In [34]:
# %%time
# import numpy as np  
# import pandas as pd  
# import matplotlib.pyplot as plt  
# from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error  
# from sklearn.model_selection import train_test_split, KFold  
# from sklearn.ensemble import GradientBoostingRegressor  
# from gensim.models.word2vec import Word2Vec  

# def fit_and_evaluate(model, X_train, y_train, X_test, y_test, filename=None, verbose=True, tolerance=0.2):  
#     # Fit the model  
#     model.fit(X_train, y_train)  

#     # Make predictions  
#     y_pred = model.predict(X_test)  

#     # Metrics  
#     r2 = r2_score(y_test, y_pred)  
#     mape = mean_absolute_percentage_error(y_test, y_pred)  
#     mse = mean_squared_error(y_test, y_pred)  

#     # Range accuracy  
#     correct_count = np.sum(np.abs(y_test - y_pred) <= tolerance * y_test)  
#     accuracy = correct_count / len(y_test)  

#     # Scatterplot  
#     if verbose:  
#         plt.scatter(y_test, y_pred)  
#         plt.xlabel('Actual Prices')  
#         plt.ylabel('Predicted Prices')  
#         plt.title('Actual vs Predicted Prices')  
#         if filename:  # Save only if filename is provided  
#             plt.savefig(filename, format='png')  
#         plt.close()  

#     if verbose:  
#         print(f"R2 Score: {r2}, MAPE: {mape}, MSE: {mse}, Accuracy: {accuracy}")  

#     return r2, mape, accuracy, mse  

# def create_word2vec_model(walks, vector_size):  
#     model = Word2Vec(walks,  
#                      hs=1,   # Hierarchical softmax  
#                      sg=1,   # Skip-gram  
#                      vector_size=vector_size,  
#                      window=5,  
#                      workers=4,  
#                      seed=1)  
#     return model  

# def get_embeddings(model, G):  
#     return np.array([model.wv[str(i)] for i in G.nodes()])  

# def grid_search_embedding_size(df_train, embedding_sizes, random_state=42):  
#     best_score = -np.inf  
#     best_params = None  

#     for size in embedding_sizes:  
#         print(f"Evaluating embedding size: {size}")  
#         G = create_mashhad_house_graph(df_train)
#         walks = generate_random_walks(G)
#         # Create Word2Vec model with the given embedding size  
#         wv_model = create_word2vec_model(walks, size)  
#         # print(G, G.nodes)
#         # Get embeddings  
#         embeddings = get_embeddings(wv_model, G)  

#         # Merge embeddings with training features  
#         embeddings_df = pd.DataFrame(embeddings, columns=[f'embedding_{i}' for i in range(embeddings.shape[1])])  
#         df_with_embeddings = pd.concat([df_train.reset_index(drop=True), embeddings_df], axis=1)  

#         # Prepare for Regression  
#         X = df_with_embeddings.drop(['price'], axis=1)  
#         y = df_with_embeddings['price']  

#         # Use cross-validation to evaluate the model  
#         kf = KFold(n_splits=5, shuffle=True, random_state=random_state)  
#         scores = []  

#         for train_index, val_index in kf.split(X):  
#             X_train_kf, X_val_kf = X.iloc[train_index], X.iloc[val_index]  
#             y_train_kf, y_val_kf = y.iloc[train_index], y.iloc[val_index]  

#             model = GradientBoostingRegressor(loss='huber', n_estimators=100, max_depth=10, random_state=random_state)  
#             r2, _, _, _ = fit_and_evaluate(model, X_train_kf, y_train_kf, X_val_kf, y_val_kf, verbose=False)  
#             scores.append(r2)  

#         mean_r2 = np.mean(scores)  
#         print(f"Mean R2 score for embedding size {size}: {mean_r2}")  

#         if mean_r2 > best_score:  
#             best_score = mean_r2  
#             best_params = size  

#     print(f"Best embedding size: {best_params} with R2: {best_score}")  
#     return best_params  

# # print(df_filtered.shape)  
# df_train, df_test = train_test_split(df_filtered, test_size=0.05, random_state=42)  

# # Perform grid search on the training set  
# embedding_sizes = [2, 3, 5, 10]#, 20, 30]  # Example sizes to test  
# best_embedding_size = grid_search_embedding_size(df_train, embedding_sizes)  
# best_embedding_size

In [21]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error  
from sklearn.model_selection import train_test_split, KFold  
from sklearn.ensemble import GradientBoostingRegressor  
# from gensim.models.word2vec import Word2Vec  

def fit_and_evaluate(model, X_train, y_train, X_test, y_test, filename=None, verbose=True, tolerance=0.2):  
    # Fit the model  
    model.fit(X_train, y_train)  

    # Make predictions  
    y_pred = model.predict(X_test)  

    # Metrics  
    r2 = r2_score(y_test, y_pred)  
    mape = mean_absolute_percentage_error(y_test, y_pred)  
    mse = mean_squared_error(y_test, y_pred)  

    # Range accuracy  
    correct_count = np.sum(np.abs(y_test - y_pred) <= tolerance * y_test)  
    accuracy = correct_count / len(y_test)  

    # Scatterplot  
    if verbose:  
        plt.scatter(y_test, y_pred)  
        plt.xlabel('Actual Prices')  
        plt.ylabel('Predicted Prices')  
        plt.title('Actual vs Predicted Prices')  
        if filename:  # Save only if filename is provided  
            plt.savefig(filename, format='png')  
        plt.close()  

    if verbose:  
        print(f"R2 Score: {r2}, MAPE: {mape}, MSE: {mse}, Accuracy: {accuracy}")  

    return r2, mape, accuracy, mse  

def create_word2vec_model(walks, vector_size):  
    model = Word2Vec(walks,  
                     hs=1,   # Hierarchical softmax  
                     sg=1,   # Skip-gram  
                     vector_size=vector_size,  
                     window=5,  
                     workers=4,  
                     seed=1)  
    return model  


def get_embeddings(model, G):  
    return np.array([model.wv[str(i)] for i in G.nodes()])  
    
def node2vec_model(df, G, vector_size):
    walks = generate_random_walks(G)
    wv_model = create_word2vec_model(walks, vector_size=vector_size)#, p=.5, q=4)  
    embeddings = get_embeddings(wv_model, G)    
    # print(node2vec_embeddings.shape)
    embeddings_df = pd.DataFrame(embeddings, columns=[f'node2vec_embedding_{i}' for i in range(embeddings.shape[1])])  
    # df_with_embeddings = pd.concat([df_filtered.reset_index(drop=True), embeddings_df], axis=1)  
    df_with_embeddings = pd.concat([df, embeddings_df], axis=1)  

    # Prepare for Regression  
    X = df_with_embeddings.drop(['price', 'id'], axis=1)  
    y = df_with_embeddings['price']  
    return X, y


def grid_search_embedding_size(df, G, embedding_sizes, random_state=42):  
    best_score = -np.inf  
    best_params = None  

    for vector_size in embedding_sizes:  
        print(f"Evaluating embedding size: {vector_size}") 
        
        X, y = node2vec_model(df, G, vector_size)
        # has_nan = X.isnull().any().any()
        # print(has_nan)  
        # print(df_filtered.shape)  
        X_train, _, y_train, _ = train_test_split(X, y, test_size=0.05, random_state=42)  
        
        # Use cross-validation to evaluate the model  
        kf = KFold(n_splits=5, shuffle=True, random_state=random_state)  
        scores = []  

        for train_index, val_index in kf.split(X_train):  
            X_train_kf, X_val_kf = X_train.iloc[train_index], X_train.iloc[val_index]  
            y_train_kf, y_val_kf = y_train.iloc[train_index], y_train.iloc[val_index]  
            
            model = GradientBoostingRegressor(loss='huber', n_estimators=100, max_depth=10, random_state=random_state)  
            r2, _, _, _ = fit_and_evaluate(model, X_train_kf, y_train_kf, X_val_kf, y_val_kf, verbose=False)  
            scores.append(r2)  

        mean_r2 = np.mean(scores)  
        print(f"Mean R2 score for embedding size {vector_size}: {mean_r2}")  

        if mean_r2 > best_score:  
            best_score = mean_r2  
            best_params = vector_size  

    print(f"Best embedding size: {best_params} with R2: {best_score}")  
    return best_params  


In [22]:
%%time

# Perform grid search on the training set  
embedding_sizes = [2, 3] #, 5]#, 10]#, 20, 30]  # Example sizes to test   
best_embedding_size = grid_search_embedding_size(df, G, embedding_sizes)  
best_embedding_size

Evaluating embedding size: 2
10000


Nodes: 100%|██████████| 10000/10000 [11:54<00:00, 13.99it/s]


Mean R2 score for embedding size 2: 0.8393298761193455
Evaluating embedding size: 3
10000


Nodes: 100%|██████████| 10000/10000 [12:26<00:00, 13.39it/s]


Mean R2 score for embedding size 3: 0.8422377491902419
Best embedding size: 3 with R2: 0.8422377491902419
CPU times: total: 36min 19s
Wall time: 30min 35s


3

In [23]:
X, y = node2vec_model(df, G, best_embedding_size)
# print(df_filtered.shape)  
X_train_w_embeddings, X_test_w_embeddings, y_train, y_test = train_test_split(X, y,\
                                                test_size=0.05, random_state=42)
X_train_wo_embeddings = X_train_w_embeddings.iloc[:, :-best_embedding_size]
X_test_wo_embeddings = X_test_w_embeddings.iloc[:, :-best_embedding_size]

10000


Nodes: 100%|██████████| 10000/10000 [12:33<00:00, 13.27it/s]


In [28]:
import pickle
# List of variables to save  
data_to_save = {  
    'X_train_wo_embeddings': X_train_wo_embeddings,  
    'X_train_w_embeddings': X_train_w_embeddings,  
    'X_test_wo_embeddings': X_test_wo_embeddings,  
    'X_test_w_embeddings': X_test_w_embeddings,  
    'y_train': y_train,  
    'y_test': y_test,  
    'best_embedding_size': best_embedding_size,  
}  

# Save the variables to a pickle file  
with open('D:/git/mamintoosi/Deepwalk-HPP/var/data_deepwalk-dist-1000.pkl', 'wb') as f:  
    pickle.dump(data_to_save, f)  

print("Variables have been saved to var folder")

Variables have been saved to var folder


In [ ]:
import pickle
# Load the variables from the pickle file  
with open('D:/git/mamintoosi/Deepwalk-HPP/var/data_deepwalk-dist-1000.pkl', 'rb') as f:  
    loaded_data = pickle.load(f)  

# Unpack the loaded data  
X_train_wo_embeddings = loaded_data['X_train_wo_embeddings']  
X_train_w_embeddings = loaded_data['X_train_w_embeddings']  
X_test_wo_embeddings = loaded_data['X_test_wo_embeddings']  
X_test_w_embeddings = loaded_data['X_test_w_embeddings']  
y_train = loaded_data['y_train']  
y_test = loaded_data['y_test']  
best_embedding_size = loaded_data['best_embedding_size']  

print("Variables have been loaded successfully!")

Variables have been loaded successfully!


## Without Word2Vec

In [24]:
model = GradientBoostingRegressor( loss = 'huber' , n_estimators=100 ,  max_depth=10 , random_state=5 )
r2, mape , accuracy,mse = fit_and_evaluate(model, X_train_wo_embeddings, \
                            y_train, X_test_wo_embeddings, y_test,'prediction_wo-deepwalk.png')

R2 Score: 0.8620527369215041, MAPE: 0.35445110262566903, MSE: 4.8744574606705e+18, Accuracy: 0.778


In [25]:
print(f"R2 Score: {r2:.2f}")
print(f"MAPE: {mape:.2f}")
print(f"RMSE: {np.sqrt(mse):.2f}")
print(f"acc: {accuracy:.2f}")
final_score = (r2 + (1 - mape) + accuracy) / 3
print(f'Final score: {final_score:.2f}')

R2 Score: 0.86
MAPE: 0.35
RMSE: 2207817352.20
acc: 0.78
Final score: 0.76


In [26]:
# Fit the model on the entire training set  
model_final = GradientBoostingRegressor(loss='huber', n_estimators=100, max_depth=10, random_state=5)  
r2_with_deepwalk, mape_with_deepwalk, accuracy_with_deepwalk, mse_with_deepwalk = fit_and_evaluate(model_final,\
            X_train_w_embeddings, y_train, X_test_w_embeddings, y_test, 'prediction_w-deepwalk.png', verbose=True)

R2 Score: 0.8734406694907636, MAPE: 0.3375801998865037, MSE: 4.4720573576524846e+18, Accuracy: 0.786


In [27]:
print(f"R2 Score: {r2_with_deepwalk:.2f}")
print(f"MAPE: {mape_with_deepwalk:.2f}")
print(f"RMSE: {np.sqrt(mse_with_deepwalk):.2f}")
print(f"Accuracy: {accuracy_with_deepwalk:.2f}")
final_score_deepwalk = (r2_with_deepwalk + (1 - mape_with_deepwalk) + accuracy_with_deepwalk) / 3
print(f'Final score: {final_score_deepwalk:.2f}')

R2 Score: 0.87
MAPE: 0.34
RMSE: 2114723943.60
Accuracy: 0.79
Final score: 0.77


In [30]:
import numpy as np  
from scipy import stats  

# این مقادیر از مدل ها به دست آمده اند.  
n_samples = X_train_w_embeddings.shape[0]  #تعداد نمونه های داده ها  
mse_without_deepwalk = mse  # MSE مدل بدون DeepWalk  
# mse_with_deepwalk = 0.8195510061449558 # MSE مدل با DeepWalk  
num_features_without = X_train_wo_embeddings.shape[1] #تعداد ویژگی های مدل بدون DeepWalk  
num_features_with = X_train_w_embeddings.shape[1]    #تعداد ویژگی های مدل با DeepWalk  
num_added_params = num_features_with - num_features_without #تعداد پارامترهای اضافه شده (10)  


# محاسبه درجات آزادی (Degrees of Freedom)  
df_without = n_samples - num_features_without -1 # برای رگرسیون خطی  
df_with = n_samples - num_features_with -1      # برای رگرسیون خطی  

#محاسبه آماره F  
F = ((mse_without_deepwalk - mse_with_deepwalk) / num_added_params) / (mse_with_deepwalk / df_with)  

# محاسبه‌ی p-value  
p_value = 1 - stats.f.cdf(F, num_added_params, df_with)  

print(f"F-statistic: {F:.2f}")  
print(f"P-value: {p_value:.3f}")  

# تفسیر نتایج  
alpha = 0.05  
if p_value < alpha:  
    print("تفاوت بین دو مدل معنی‌دار است (p-value < 0.05).")  
else:  
    print("تفاوت بین دو مدل معنی‌دار نیست (p-value >= 0.05).")  

F-statistic: 284.04
P-value: 0.000
تفاوت بین دو مدل معنی‌دار است (p-value < 0.05).
